# Slurk Log Viewer

In [224]:
#import packages
import os
import json
import requests
import configparser
import sys
from datetime import datetime

## Choose Session (optional)

In [225]:
CONFIG = configparser.ConfigParser()
CONFIG.read('config.ini')
token = CONFIG['logs']['admin_token']
uri = CONFIG['logs']['url'] # "http://localhost/api/v2"
SESSION = CONFIG['session']['name'] #  "Test_local"

print("Session: " + SESSION)
if not os.path.isdir('./logs/' + SESSION):
    os.mkdir('./logs/' + SESSION)
session_folder = os.path.join("./logs", SESSION)

Session: session_240919_1


## List Available Rooms & download logs (optional)

In [226]:
rooms = requests.get(f"{uri}/rooms", headers={'Authorization': f"Token {token}"})
if not rooms.ok:
    print("Could not get rooms")
    sys.exit(3)

# go over each cola room log #
for room in rooms.json():
    room_name = room["name"]
    print(room_name)
    logs = requests.get(f"{uri}/room/{room_name}/logs", headers={'Authorization': f"Token {token}"})
    if not logs.ok:
        print("Could not get logs")
        sys.exit(4)


    with open('./logs/' + SESSION + '/'+ room_name +'.json', 'w') as outfile:
        json.dump(logs.json(), outfile)

admin_room
waiting_room
cola-3e9262f6-de52-11e9-b351-000c29373007
cola-eb5a4224-de52-11e9-b351-000c29373007
cola-0ac06fb2-de53-11e9-b351-000c29373007
cola-1592fbbc-de53-11e9-b351-000c29373007
cola-618e2022-de54-11e9-b351-000c29373007
cola-58c8c2d8-de56-11e9-b351-000c29373007
cola-82f6d964-de56-11e9-b351-000c29373007
cola-95320590-de56-11e9-b351-000c29373007
cola-ac54d9f0-de56-11e9-b351-000c29373007


## List Log Files (optional)

In [231]:
onlyfiles = [f for f in os.listdir(session_folder) if os.path.isfile(os.path.join(session_folder, f))]
for log_file in onlyfiles:
    print(os.path.join(session_folder, log_file))

./logs/session_240919_1/cola-3e9262f6-de52-11e9-b351-000c29373007.json
./logs/session_240919_1/cola-1592fbbc-de53-11e9-b351-000c29373007.json
./logs/session_240919_1/cola-ac54d9f0-de56-11e9-b351-000c29373007.json
./logs/session_240919_1/cola-82f6d964-de56-11e9-b351-000c29373007.json
./logs/session_240919_1/cola-0ac06fb2-de53-11e9-b351-000c29373007.json
./logs/session_240919_1/waiting_room.json
./logs/session_240919_1/admin_room.json
./logs/session_240919_1/cola-618e2022-de54-11e9-b351-000c29373007.json
./logs/session_240919_1/cola-eb5a4224-de52-11e9-b351-000c29373007.json
./logs/session_240919_1/cola-58c8c2d8-de56-11e9-b351-000c29373007.json
./logs/session_240919_1/cola-95320590-de56-11e9-b351-000c29373007.json


## Show Logs

In [233]:
log_file = "./logs/session_240919_1/cola-95320590-de56-11e9-b351-000c29373007.json"
print("Log: " + log_name)

with open(log_file) as json_file:
    logs = json.load(json_file)

# go over each type of chat - process text and commands
# and write them to a file.
for log_entry in logs:
    starter_str = str(datetime.fromtimestamp(log_entry['date_created'])) \
                  + " - " \
                  + log_entry['user']['name'] \
                  + " - " \
                  + log_entry['room']\
                  + ' - ' \
                  + log_entry['event'] \
                  + " - "
    if log_entry['event'] == "text_message":
        chat_str = starter_str + log_entry['message'].replace('\n', ' ')
    elif log_entry['event'] == 'command':
        if log_entry["command"].startswith('answer'):
            data_str = "".join(log_entry["data"]["command"].split("answer")[1:]).strip()
            chat_str = starter_str + 'answer: '+ data_str
        elif log_entry["command"] == 'agree':
            data_str = ' '.join(log_entry['data'])
            chat_str = starter_str + 'agree'
        elif log_entry['command'] == 'ready':
            chat_str = starter_str + 'ready'
    elif log_entry['event'] == 'join':
        chat_str = starter_str 
    elif log_entry['event'] == 'leave':
        chat_str = starter_str
    elif log_entry['event'] == 'set_attribute':
        chat_str = starter_str + log_entry['id'] + ' - ' + log_entry['attribute'] + ' - '+ log_entry['value']
    elif log_entry['event'] == 'set_text':
        chat_str = starter_str + log_entry['text'].replace('\n', ' ')
    elif log_entry['event'] == 'confirmation_log':
        chat_str = starter_str + log_entry['status_txt']
    else:
        print(log_entry)

    print(chat_str)

Log: cola-1592fbbc-de53-11e9-b351-000c29373007.json
2019-09-24 01:05:06 - frightened - cola-95320590-de56-11e9-b351-000c29373007 - join - 
2019-09-24 01:05:06 - petite - cola-95320590-de56-11e9-b351-000c29373007 - join - 
2019-09-24 01:05:06 - Cola Bot - cola-95320590-de56-11e9-b351-000c29373007 - join - 
2019-09-24 01:05:06 - Cola Bot - cola-95320590-de56-11e9-b351-000c29373007 - text_message -  **Welcome to the CoLa Game!** Discussion and providing reason(s) for your answer is crucial for this game.
2019-09-24 01:05:06 - Cola Bot - cola-95320590-de56-11e9-b351-000c29373007 - text_message -  Remember the following commands to play the game:   (1) Propose answer to your partner: Type "/answer ...your description here...".   (2) Agree on the answer proposed by your partner: Type "/agree".
2019-09-24 01:05:06 - Cola Bot - cola-95320590-de56-11e9-b351-000c29373007 - text_message - Please type **/ready** to begin the game.
2019-09-24 01:05:18 - frightened - cola-95320590-de56-11e9-b351-000